# Performative Optimization: Credit Score Experiments

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_debug_nans", True)

from data_generation import StratClassLinearDGP
import algorithms

%matplotlib inline

In [ ]:
def fit_model_based(X, Y, response_model, lam, diameter, tol=1e-10, n_steps=1000):
    
    n, d = X.shape
    
    def logistic_loss(theta, X, Y, response_model, lam):
        perf_effects = jnp.dot(response_model, theta)
        X_strat = X + perf_effects
        preds = jnp.dot(X_strat, theta)
        t1 = jnp.mean(-1.0 * Y * preds + jnp.log1p(jnp.exp(preds)))
        t2 = lam / 2.0 * jnp.sum(jnp.square(theta[:-1])) # last coordinate is the bias
        return t1 + t2
    
    jax_grad = jax.grad(logistic_loss)

    eta_init = 1
   
    theta = jnp.zeros(d)

    # evaluate initial loss
    prev_loss = logistic_loss(theta, X, Y, response_model, lam)
    loss_list = [prev_loss]

    gap = 1e30
    eta = eta_init
    count = 0

    while count < n_steps:

        if gap < tol:
            break

        gradient = jax_grad(theta, X, Y, response_model, lam)
        theta_new = theta - eta * gradient
        
#         projection step
        norm = np.linalg.norm(theta_new)
        if norm >= diameter:
            print('projected')
            theta_new *= diameter / norm 
        
        if eta < 1e-10:
            break

        # compute new loss
        loss = logistic_loss(theta_new, X, Y, response_model, lam)

        # do backtracking line search
        if loss > prev_loss:
            eta = eta * .5
            gap = 1e30
            continue
        else:
            eta = eta_init

        theta = np.copy(theta_new)

        loss_list.append(loss)

        gap = prev_loss - loss
        prev_loss = loss

        count = count + 1
        

    return theta

In [ ]:
path_to_csv_file = './kaggle_data/cs-training.csv'
strat_features = np.array([1,6,8]) - 1 # for later

input_dim = 11
sigma_exp = 1
eps = 0.0001 # use for eps < gamma/2beta
# eps = 100 # use for large epsilon
diameter = 10
n_steps = 1000
lam = .002
trials = 50
step = 10000 # evaluate loss and accuracy after every "step" number of samples



for num_train in [1000000]:
    
    model_based_loss = []
    model_based_acc = []
    greedy_loss = []
    greedy_acc = []
    lazy_loss = []
    lazy_acc = []
    dfo_loss = []
    dfo_acc = []
     
    print('\n Num Train {}'.format(num_train))
    
    dgp = StratClassLinearDGP(path_to_csv_file, eps, strat_features, lam)
    
    # compute problem parameters as per the theory
    N = dgp.num_points
    max_squared_feature_norm = 2 * np.sum(np.square(dgp.features)) + \
                               2 * (np.linalg.norm(dgp.update_matrix) * diameter) ** 2 
    L =  max_squared_feature_norm / (4.0 * N) + lam 
    gamma = lam
    beta = max(2, L)
    
    if eps < gamma / (2*beta): 
        print("eps < gamma / (2*beta)")
    
    if eps < gamma / beta: 
        print("eps < gamma / (beta)")
        greedy_c =  1 / (gamma - eps * beta)
        greedy_k0 = 8 * (L / (gamma - eps * beta)) ** 2
    else:
        print("eps >= gamma / beta")
        greedy_c = 100 / gamma
        greedy_k0 = 8 * (L / gamma) ** 2 
        
    for trial in range(trials):
        
        print(trial)
   
        # model based
        model_based_loss_tmp = [dgp.population_loss(np.zeros(input_dim))]
        model_based_acc_tmp = [dgp.population_accuracy(np.zeros(input_dim))]
    
        for model_based_samples in np.arange(step,num_train+1,step):
            
            random_thetas = np.random.randn(model_based_samples // 2, input_dim)
            X_exp, Y_exp = dgp.generate_data(random_thetas)

            response_model, _ , _, _ = jnp.linalg.lstsq(random_thetas, X_exp)

            zeros = np.zeros((model_based_samples // 2, input_dim))
            X_train, Y_train = dgp.generate_data(zeros)

            model_based_theta = fit_model_based(X_train, Y_train, 
                                                                       response_model, lam, diameter, tol=1e-10)
            model_based_loss_tmp.append(dgp.population_loss(model_based_theta))
            model_based_acc_tmp.append(dgp.population_accuracy(model_based_theta))


        model_based_loss.append(model_based_loss_tmp)
        model_based_acc.append(model_based_acc_tmp)



        # Greedy SGD
        greedy_theta_list = algorithms.greedy_sgd(dgp, num_train, diameter, greedy_k0, greedy_c)
        
        greedy_loss.append([dgp.population_loss(np.zeros(input_dim))] + [dgp.population_loss(greedy_theta_list[i]) for i in np.arange(step-1,len(greedy_theta_list),step)])
        greedy_acc.append([dgp.population_accuracy(np.zeros(input_dim))] + [dgp.population_accuracy(greedy_theta_list[i]) for i in np.arange(step-1,len(greedy_theta_list),step)])

        

        # Lazy SGD
        lazy_c =  1 / gamma
        lazy_k0 = 8 * (L / gamma) ** 2
        alpha = 2

        lazy_theta_list = algorithms.lazy_sgd(dgp, num_train, diameter, lazy_k0, lazy_c, alpha)
        
        lazy_loss.append([dgp.population_loss(np.zeros(input_dim))] + [dgp.population_loss(lazy_theta_list[i]) for i in np.arange(step-1,len(lazy_theta_list),step)])
        lazy_acc.append([dgp.population_accuracy(np.zeros(input_dim))] + [dgp.population_accuracy(lazy_theta_list[i]) for i in np.arange(step-1,len(lazy_theta_list),step)])

        
        # DFO algorithm
        samples_to_avg = 100
        step_sequence = [1/(k) for k in range(1, round(num_train/samples_to_avg))]

        dfo_theta_list = flaxman_dfo(dgp, step_sequence, diameter, samples_to_avg)

        step_dfo = round(step/samples_to_avg)
        dfo_loss.append([dgp.population_loss(np.zeros(input_dim))] + [dgp.population_loss(dfo_theta_list[i-1]) for i in np.arange(step_dfo,len(dfo_theta_list)+1,step_dfo)])
        dfo_acc.append([dgp.population_accuracy(np.zeros(input_dim))] + [dgp.population_accuracy(dfo_theta_list[i-1]) for i in np.arange(step_dfo,len(dfo_theta_list)+1,step_dfo)])




    

    

In [ ]:
data = []
for step in range(101):
    for trial in range(50):
        data.append({"step": step*10000, "algorithm": 'Greedy SGD',
                         "loss": greedy_loss[trial][step],
                        "accuracy": greedy_acc[trial][step]})
        
        data.append({"step": step*10000, "algorithm": 'Lazy SGD',
                         "loss": lazy_loss[trial][step],
                        "accuracy": lazy_acc[trial][step]})
        
        data.append({"step": step*10000, "algorithm": 'Flaxman DFO',
                         "loss": dfo_loss[trial][step],
                        "accuracy": dfo_acc[trial][step]})
        
        data.append({"step": step*10000, "algorithm": 'Model-Based',
                         "loss": model_based_loss[trial][step],
                        "accuracy": model_based_acc[trial][step]})

In [ ]:
import json

with open('strat_sim_data.json', 'w') as outfile:
    json.dump(data, outfile)